<a href="https://colab.research.google.com/github/akincemtutal9/Fish/blob/main/Drug_Target_interaction_prediction_through%C2%A0Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Drug-Target interaction prediction through Python

Please visit the article on Medium about detail instruction

#### Download drug-target interaction flat file from DrugCentral

In [1]:
import os
import itertools
import pandas as pd

In [3]:
dti_file_url = r'https://unmtid-shinyapps.net/download/DrugCentral/2021_09_01/drug.target.interaction.tsv.gz'
df_dti = pd.read_csv(dti_file_url, compression='gzip', sep='\t')

In [4]:
df_dti.head(3)

,DRUG_NAME,STRUCT_ID,TARGET_NAME,TARGET_CLASS,ACCESSION,GENE,SWISSPROT,ACT_VALUE,ACT_UNIT,ACT_TYPE,ACT_COMMENT,ACT_SOURCE,RELATION,MOA,MOA_SOURCE,ACT_SOURCE_URL,MOA_SOURCE_URL,ACTION_TYPE,TDL,ORGANISM
0,levobupivacaine,4,Potassium voltage-gated channel subfamily H me...,Ion channel,Q12809,KCNH2,KCNH2_HUMAN,4.89,NaN,IC50,Inhibition of wild-type human ERG channel expr...,CHEMBL,=,NaN,NaN,NaN,NaN,NaN,Tclin,Homo sapiens
1,levobupivacaine,4,Sodium channel protein type 1 subunit alpha,Ion channel,P35498,SCN1A,SCN1A_HUMAN,5.79,NaN,IC50,NaN,WOMBAT-PK,=,NaN,NaN,NaN,NaN,NaN,Tclin,Homo sapiens
2,levobupivacaine,4,Sodium channel protein type 4 subunit alpha,Ion channel,P35499,SCN4A,SCN4A_HUMAN,NaN,NaN,NaN,NaN,WOMBAT-PK,NaN,1.0,CHEMBL,NaN,https://www.ebi.ac.uk/chembl/compound/inspect/...,BLOCKER,Tclin,Homo sapiens


In [5]:
f'Row number (Drug-Target link): {df_dti.shape[0]}, Column number (Metadata): {df_dti.shape[1]}'

'Row number (Drug-Target link): 19378, Column number (Metadata): 20'

In [6]:
f'Unique drug number: {len(df_dti.DRUG_NAME.drop_duplicates())}'

'Unique drug number: 2587'

In [7]:
f'Unique target accession number: {len(set(list(itertools.chain(*[str(x).split("|") for x in df_dti.ACCESSION]))))}'

'Unique target accession number: 3179'

#### Generate a similarity matrix using Jaccard distance

In [8]:
import urllib

In [9]:
raw_folder = r'https://raw.githubusercontent.com/luoyunan/DTINet/master/data/'
raw_files = {'drug_drug': 'mat_drug_drug.txt', 'drug_disease': 'mat_drug_disease.txt',
             'drug_sideeffect': 'mat_drug_se.txt', 'protein_disease': 'mat_protein_disease.txt',
             'protein_protein': 'mat_protein_protein.txt'}

raw_df = dict()
for key, val in raw_files.items():   
    raw_df[key] = pd.read_csv(urllib.parse.urljoin(raw_folder, val), header=None, sep='\s+')
    print(f'{key} table shape: {raw_df[key].shape}')

drug_drug table shape: (708, 708)
drug_disease table shape: (708, 5603)
drug_sideeffect table shape: (708, 4192)
protein_disease table shape: (1512, 5603)
protein_protein table shape: (1512, 1512)


In [10]:
# add drug code, disease name
drug_name = pd.read_csv(urllib.parse.urljoin(raw_folder,'drug.txt'), header=None)
disease_name = pd.read_csv(urllib.parse.urljoin(raw_folder,'disease.txt'), header=None, sep='\t')

raw_df['drug_disease'].index = drug_name[0]; raw_df['drug_disease'].index.name =None
raw_df['drug_disease'].columns = disease_name[0]
raw_df['drug_disease'].iloc[:5,:5]

,depressive disorder,drug-induced liver injury,mercury poisoning,necrosis,neoplasms
DB00050,1,1,1,1,1
DB00152,1,1,0,0,1
DB00162,1,1,0,1,1
DB00175,1,1,0,1,1
DB00176,1,1,0,1,0


In [11]:
import numpy as np
from sklearn.metrics import pairwise_distances

In [12]:
# Merge each drug and target similarity files
similarity_df = dict()
for key, val in raw_df.items():
    _df = val.astype(bool).to_numpy()
    _df = pd.DataFrame(pairwise_distances(_df, metric='jaccard'))
    _df = 1 - _df.applymap(lambda x: 1 if x == 0 else x)
    similarity_df[key] = _df + np.eye(len(_df))

similarity_df['drug_disease'].apply(lambda x: round(x,2))

,0,1,2,3,4,5,6,7,8,9,...,698,699,700,701,702,703,704,705,706,707
0,1.00,0.19,0.29,0.37,0.19,0.42,0.37,0.10,0.24,0.32,...,0.01,0.01,0.02,0.02,0.24,0.00,0.02,0.0,0.32,0.35
1,0.19,1.00,0.13,0.20,0.16,0.19,0.27,0.10,0.18,0.14,...,0.01,0.01,0.04,0.01,0.14,0.00,0.02,0.0,0.22,0.20
2,0.29,0.13,1.00,0.45,0.16,0.45,0.30,0.05,0.22,0.41,...,0.00,0.01,0.02,0.01,0.13,0.00,0.02,0.0,0.26,0.33
3,0.37,0.20,0.45,1.00,0.22,0.49,0.40,0.07,0.30,0.44,...,0.00,0.02,0.02,0.01,0.16,0.00,0.02,0.0,0.38,0.57
4,0.19,0.16,0.16,0.22,1.00,0.17,0.17,0.08,0.23,0.24,...,0.00,0.02,0.03,0.04,0.17,0.00,0.04,0.0,0.20,0.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
703,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,1.00,0.06,0.0,0.00,0.00
704,0.02,0.02,0.02,0.02,0.04,0.02,0.02,0.00,0.04,0.02,...,0.00,0.00,0.00,0.00,0.01,0.06,1.00,0.0,0.03,0.02
705,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.01,0.00,0.00,1.0,0.00,0.00
706,0.32,0.22,0.26,0.38,0.20,0.37,0.31,0.10,0.42,0.35,...,0.01,0.02,0.04,0.02,0.19,0.00,0.03,0.0,1.00,0.50


#### Reduce similarity matrix dimensions using SVD

In [13]:
similarity_df['drug'] = pd.read_csv(urllib.parse.urljoin(raw_folder,'Similarity_Matrix_Drugs.txt'), 
                                    header=None, sep='\s+')
similarity_df['protein'] = pd.read_csv(urllib.parse.urljoin(raw_folder,'Similarity_Matrix_Proteins.txt'), 
                                       header=None, sep=' ')

In [14]:
def calc_dca(df_mat, maxiter=20, restart_prob=0.5):
    
    mat = df_mat.div(df_mat.sum(axis=1),axis=0).to_numpy()
    restart = np.eye(len(mat))
    mat_ret = np.eye(len(mat))
    
    for i in range(maxiter):
        mat_ret_new = (1-restart_prob) * mat * mat_ret + restart_prob * restart
        delta = np.linalg.norm(mat_ret-mat_ret_new, ord='fro')
        mat_ret = mat_ret_new
        if delta < 1e-6:
            break 
            
    return mat_ret

In [15]:
drug_mat = ['drug_drug','drug_disease','drug_sideeffect','drug']
protein_mat = ['protein_disease','protein_protein','protein']

drug_dca = np.concatenate([calc_dca(similarity_df[key]) for key in drug_mat], axis=1)
protein_dca = np.concatenate([calc_dca(similarity_df[key]) for key in protein_mat], axis=1)

# Concatenate matrix of Drug similarity
pd.DataFrame(drug_dca).apply(lambda x: round(x, 2))

,0,1,2,3,4,5,6,7,8,9,...,2822,2823,2824,2825,2826,2827,2828,2829,2830,2831
0,1.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.66,0.00,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.00,0.54,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.00,0.00,0.51,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
703,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0
704,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0
705,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0
706,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0


In [16]:
from sklearn.decomposition import TruncatedSVD

def calc_svd(dca_mtx, num_feature=100):
    
    alpha = 1 / len(dca_mtx)
    dca_mtx = np.log(dca_mtx + alpha) - np.log(alpha)
    dca_mtx = np.matmul(dca_mtx, dca_mtx.transpose())

    svd = TruncatedSVD(n_components=num_feature)
    svd.fit(dca_mtx)
    result = svd.transform(dca_mtx)
    
    return result

In [17]:
# number of features
num_feature = 100

drug_svd = calc_svd(drug_dca, num_feature)
protein_svd = calc_svd(protein_dca, num_feature)

pd.DataFrame(drug_svd).apply(lambda x: round(x,2))

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-4.64,-0.09,-12.64,-7.25,15.45,2.60,21.83,-9.13,-6.68,3.86,...,2.21,-4.26,-6.12,7.15,8.93,-1.38,0.03,-0.88,0.48,-4.12
1,0.53,-6.26,-2.77,10.65,3.32,0.59,6.73,-8.37,-7.18,5.01,...,0.22,-0.55,1.46,-0.10,-2.14,-4.41,-0.64,-2.01,-8.06,-5.55
2,-5.17,6.35,4.41,2.01,-2.76,0.45,-4.32,-4.03,-1.99,2.00,...,-1.37,10.94,-10.51,1.01,-7.73,-6.68,0.66,-4.60,7.69,-5.25
3,-1.77,0.96,-3.58,0.85,4.05,-0.44,-0.12,1.32,-1.90,2.09,...,3.57,-1.41,9.55,-3.00,-11.86,-1.53,4.62,-10.46,-3.98,-0.14
4,0.91,-3.05,-1.37,-1.42,1.82,1.75,3.43,1.41,1.87,-0.63,...,-1.83,4.76,-3.32,2.62,-7.60,2.06,-0.86,3.89,0.90,-5.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
703,-0.92,-18.11,4.23,45.92,49.40,-15.66,-11.05,1.34,19.55,0.51,...,-6.53,0.13,0.58,2.78,-2.06,1.86,3.54,-0.75,-2.41,2.94
704,-5.62,-8.74,9.37,5.40,1.08,-8.55,7.32,14.51,-27.50,-17.93,...,2.16,-0.90,1.60,1.21,1.85,2.87,5.64,-4.81,-4.50,4.91
705,4.03,21.36,8.87,8.84,21.09,26.86,12.62,14.16,0.19,23.35,...,3.37,-0.83,1.03,-0.46,-3.48,-6.19,4.46,-4.67,2.79,-1.47
706,4.19,-0.89,-1.11,-1.64,-0.03,5.03,3.67,-0.01,3.12,4.97,...,-13.32,-6.75,-9.27,-1.96,1.57,-6.56,-3.96,-10.15,-4.89,5.31


#### Matrix completion to search new drug-target interaction

In [18]:
import time
from numba import jit

In [19]:
@jit(nopython=True)
def matrix_factorization(R, P, Q, K, steps=5000, alpha=0.0002, beta=0.02):
    '''
    ORIGINAL SOURCE: https://towardsdatascience.com/recommendation-system-matrix-factorization-d61978660b4b
    R: rating matrix
    P: |U| * K (User features matrix)
    Q: |D| * K (Item features matrix)
    K: latent features
    steps: iterations
    alpha: learning rate
    beta: regularization parameter'''
    Q = Q.T

    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    # calculate error
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j])

                    for k in range(K):
                        # calculate gradient with a and beta parameter
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])

        eR = np.dot(P,Q)

        e = 0

        for i in range(len(R)):

            for j in range(len(R[i])):

                if R[i][j] > 0:

                    e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2)

                    for k in range(K):

                        e = e + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))
        # 0.001: local minimum
        if e < 0.001:

            break

    return P, Q.T

In [20]:
R = pd.read_csv(urllib.parse.urljoin(raw_folder,'mat_drug_protein.txt'), header=None, sep='\s+').to_numpy()

# N: number of drugs
N = R.shape[0]
# M: number of proteins
M = R.shape[1]
# Number of feature
K = num_feature

start = time.time()
n_drug, n_protein = matrix_factorization(R, drug_svd, protein_svd, K)
end = time.time()
print(f'Elapse: {end-start}')

Elapse: 190.58936619758606


In [21]:
n_drug_protein = np.dot(n_drug, n_protein.T)

#### Evaluate predicted drug-target interaction

In [22]:
from sklearn.metrics import roc_auc_score

In [23]:
rhat = n_drug_protein
rhat = pd.DataFrame(rhat).stack().reset_index()
rhat.set_index(rhat.apply(lambda x: f'{int(x.level_0)}_{int(x.level_1)}', axis=1), inplace=True)
rhat = rhat[0]
rhat.sort_index(inplace=True)

r = R
r = pd.DataFrame(r).stack().reset_index()
r.set_index(r.apply(lambda x: f'{int(x.level_0)}_{int(x.level_1)}', axis=1), inplace=True)
r = r[0]
r = r[rhat.index]
r.sort_index(inplace=True)

roc_auc_score(r, rhat)

0.5025264794480614